In [8]:
import pandas as pd
import sqlite3 as sq
from datetime import datetime

In [18]:
# Load the 'price_action' table from the 'market_data.db' database into a DataFrame named 'price_action'.
conn = sq.connect('../market_data.db')
price_action = pd.read_sql_query("SELECT * FROM price_action", conn)
conn.close()

# Load the mkt_cap_next_earnings table from the database
conn = sq.connect('../market_data.db')
mkt_cap_data = pd.read_sql_query("SELECT * FROM mkt_cap_next_earnings", conn)
conn.close()

In [11]:
# Define the Minervini criteria with the new eliminatory criteria
def minervini_criteria_full(row):
    criteria = [
        row['Price'] > 5,
        row['Price'] < 300,
        row['Price'] > row['SMA50'],
        row['SMA50'] > row['SMA150'],
        row['SMA150'] > row['SMA200'],
        row['Price'] >= row['52Week-Low'] * 1.25,
        row['Price'] >= row['52Week-High'] * 0.85,
        row['RS Rating'] >= 70,
        row['RS_Rating_6W_Trend'] == 1,
        row['SMA200_1M_Trend'] == 1,
        row['Avg_Vol_90d'] > 100000,
        row['Avg_Vol_10d'] <= row['Avg_Vol_30d']

    ]
    return 'yes' if all(criteria) else 'no'

In [12]:
# Define the Minervini criteria with the new eliminatory criteria
def minervini_criteria_basic(row):
    criteria = [
        row['Price'] > 5,
        row['Price'] < 300,
        row['Price'] > row['SMA50'],
        row['SMA50'] > row['SMA150'],
        row['SMA150'] > row['SMA200'],
        row['RS Rating'] >= 70,
        row['Avg_Vol_90d'] > 100000,

    ]
    return 'yes' if all(criteria) else 'no'

In [13]:
# Define the PowerTrend criteria
def power_trend(row, prev_power_trend):
    # Entry criteria
    if (
        row['lows_above_EMA21'] >= 10 and
        row['EMA21_above_SMA50'] >= 5 and
        row['SMA50_uptrend'] == 1 and
        row['close_above_open'] == 1
    ):
        return 'in'
    
    # Exit criteria
    if prev_power_trend == 'in' and (
        row['EMA21'] < row['SMA50'] or
        (row['close'] < row['high_last_week'] * 0.9 and row['close'] < row['SMA50'])
    ):
        return 'out'
    
    return prev_power_trend

In [14]:
# Add the 'Price' column representing the latest close for each stock
price_action['Price'] = price_action.groupby('ticker')['close'].transform('last')

# Calculate the 50-day Simple Moving Average (SMA50) for each stock.
price_action['SMA50'] = price_action.groupby('ticker')['close'].transform(lambda x: x.rolling(window=50).mean())

# Calculate the 21-day Exponential Moving Average (EMA21) for each stock.
price_action['EMA21'] = price_action.groupby('ticker')['close'].transform(lambda x: x.ewm(span=21, adjust=False).mean())

# Calculate the 150-day Simple Moving Average (SMA150) for each stock.
price_action['SMA150'] = price_action.groupby('ticker')['close'].transform(lambda x: x.rolling(window=150).mean())

# Calculate the 200-day Simple Moving Average (SMA200) for each stock.
price_action['SMA200'] = price_action.groupby('ticker')['close'].transform(lambda x: x.rolling(window=200).mean())

# Calculate the 90-day Average Volume for each stock.
price_action['Avg_Vol_90d'] = price_action.groupby('ticker')['volume'].transform(lambda x: x.rolling(window=90).mean())

# Calculate the 30-day Average Volume for each stock.
price_action['Avg_Vol_30d'] = price_action.groupby('ticker')['volume'].transform(lambda x: x.rolling(window=30).mean())

# Calculate the 10-day Average Volume for each stock.
price_action['Avg_Vol_10d'] = price_action.groupby('ticker')['volume'].transform(lambda x: x.rolling(window=10).mean())

# Calculate the 52-week high (highest closing price in the last 252 trading days) for each stock.
price_action['52Week-High'] = price_action.groupby('ticker')['close'].transform(lambda x: x.rolling(window=252).max())

# Calculate the 52-week low (lowest closing price in the last 252 trading days) for each stock.
price_action['52Week-Low'] = price_action.groupby('ticker')['close'].transform(lambda x: x.rolling(window=252).min())

# Calculate the price performance over the last 252 trading days
price_action['12M Return'] = price_action.groupby('ticker')['close'].pct_change(periods=252)

# Calculate the RS Rating based on the 12M Return percentile rank
price_action['RS Rating'] = price_action['12M Return'].rank(pct=True) * 100

# Calculate the highest high and lowest low over the last 5 days
price_action['10D_high'] = price_action.groupby('ticker')['high'].transform(lambda x: x.rolling(window=10).max())
price_action['10D_low'] = price_action.groupby('ticker')['low'].transform(lambda x: x.rolling(window=10).min())

# Calculate the range as a percentage of the highest high
price_action['10D_range_pct'] = (price_action['10D_high'] - price_action['10D_low']) / price_action['10D_high'] * 100

# Identify low-risk entries
price_action['Low-Risk_Entry'] = (price_action['10D_range_pct'] < 8).astype(int)

# Calculate the RS Rating trend over the last 6 weeks and SMA200 trend over the last month
price_action['RS_Rating_6W_Trend'] = price_action.groupby('ticker')['RS Rating'].transform(lambda x: x.diff(periods=30)).gt(0).astype(int)
price_action['SMA200_1M_Trend'] = price_action.groupby('ticker')['SMA200'].transform(lambda x: x.diff(periods=21)).gt(0).astype(int)

# Calculate criteria for PowerTrend
price_action['lows_above_EMA21'] = price_action.groupby('ticker').apply(lambda x: (x['low'] > x['EMA21']).rolling(window=10).sum()).reset_index(level=0, drop=True)
price_action['EMA21_above_SMA50'] = price_action.groupby('ticker').apply(lambda x: (x['EMA21'] > x['SMA50']).rolling(window=5).sum()).reset_index(level=0, drop=True)
price_action['SMA50_uptrend'] = price_action.groupby('ticker')['SMA50'].transform(lambda x: x.diff().rolling(window=21).sum()).gt(0).astype(int)
price_action['close_above_open'] = (price_action['close'] > price_action['open']).astype(int)
price_action['high_last_week'] = price_action.groupby('ticker')['high'].transform(lambda x: x.rolling(window=5).max())

# Apply the Minervini criteria to each row
price_action['Minervini_basic'] = price_action.apply(minervini_criteria_basic, axis=1)
price_action['Minervini_full'] = price_action.apply(minervini_criteria_full, axis=1)

# Apply the PowerTrend criteria to each row
prev_power_trend = 'out'
power_trends = []

for _, row in price_action.iterrows():
    current_power_trend = power_trend(row, prev_power_trend)
    power_trends.append(current_power_trend)
    prev_power_trend = current_power_trend
    
# Apply the Minervini criteria to each row
price_action['PowerTrend'] = power_trends

# Create the 'screen_df' DataFrame with the calculated metrics.
metrics = ['ticker', 'EMA21', 'SMA50', 'SMA150', 'SMA200', 'Avg_Vol_10d', 'Avg_Vol_30d', 'Avg_Vol_90d', '52Week-High', '52Week-Low', 'Minervini_basic', 'Minervini_full', 'PowerTrend', 'Low-Risk_Entry']
screen_df = price_action.drop_duplicates(subset='ticker', keep='last')[metrics]
screen_df = pd.merge(screen_df, mkt_cap_data[['Ticker', 'Mkt Cap', 'Next Earnings Date']], left_on='ticker', right_on='Ticker', how='left')
screen_df.drop('Ticker', axis=1, inplace=True)
screen_df

,ticker,EMA21,SMA50,SMA150,SMA200,Avg_Vol_10d,Avg_Vol_30d,Avg_Vol_90d,52Week-High,52Week-Low,Minervini_basic,Minervini_full,PowerTrend,Low-Risk_Entry
386526,ARYE,10.356026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,no,in,0
644568,BGRY,1.400555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,no,in,0
664240,BICK,27.597501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,no,in,0
664710,BIDS,14.374607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,no,out,0
738294,BLU,14.645106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,no,in,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6928789,ZVRA,5.026390,5.10580,5.266367,5.135175,128705.9,1.246986e+05,2.508118e+05,6.850000,4.120000,no,no,out,0
6928790,ZWS,29.000115,27.84660,23.891600,23.682100,835586.9,1.079376e+06,1.370451e+06,30.639999,19.790001,no,no,out,1
6928791,ZYME,7.554345,7.87240,8.598800,8.378100,437161.4,5.562705e+05,5.759346e+05,10.220000,4.650000,no,no,out,0
6928792,ZYNE,0.814084,0.47982,0.457653,0.489420,3391755.2,1.231035e+06,5.419584e+05,1.350000,0.300000,no,no,out,0


In [15]:
today = datetime.now().strftime('%Y-%m-%d')
nome_arquivo = f'../output/screen_{today}.xlsx'
screen_df.to_excel(nome_arquivo, engine='xlsxwriter')

In [16]:
# Get the current date
current_date = datetime.now().strftime('%d-%m-%Y')
table_name = f"screen-{current_date}"

# Connect to the SQLite database
conn = sq.connect('../market_data.db')

# Save the screen_df to the database with the table name based on the current date
screen_df.to_sql(table_name, conn, if_exists='replace', index=False)

# Close the database connection
conn.close()